## get results info

In [1]:
from pathlib import Path
import pickle
import requests

from collections import defaultdict

from geoloc_imc_2023.helpers import rtt_to_km, distance

measurement_file = Path(".") / "../datasets/test_geoloc/10_probes_10_target.pickle"
probe_atlas_file = "../datasets/probes_atlas.pickle"

with open(probe_atlas_file, "rb") as f:
    probes_atlas = pickle.load(f)

anchors = {}
for probe, probe_description in probes_atlas.items():
    if probe_description['is_anchor']:
        anchors[probe] = probe_description


# get measurement results
with open(measurement_file, "rb") as f:
    measurement_results = pickle.load(f)

geoloc_results = defaultdict(dict)

def get_from_atlas(url):
    """request atlas api"""
    response = requests.get(url).json()
    for result in response:
        yield result

        # if response["next"]:
        #     response = requests.get(response["next"]).json()
        # else:
        #     break

key = "b3d3d4fc-724e-4505-befe-1ad16a70dc87"
for target_ip in measurement_results:
    measurement_id = measurement_results[target_ip]['measurement_id']
    url = f"https://atlas.ripe.net/api/v2/measurements/{measurement_id}/results/?key={key}"

    results = get_from_atlas(url)

    print(f"getting results for target ip: {target_ip}")

    for result in results:
        # parse results and calculate geoloc
        if result.get('result') is not None:
            try:
                vp_ip = result['from']
                min_rtt =  min([list(rtt.values())[0] for rtt in result['result']])

                # both vp and target coordinates
                vp_lat = anchors[vp_ip]['latitude']
                vp_lon = anchors[vp_ip]['longitude']
                target_lat = anchors[target_ip]["latitude"]
                target_lon = anchors[target_ip]["longitude"]

                # save measurement results for each pair target <-> vp
                geoloc_results[target_ip][vp_ip] = {}
                geoloc_results[target_ip][vp_ip]['rtt'] = min_rtt
                geoloc_results[target_ip][vp_ip]["latitude"]= vp_lat
                geoloc_results[target_ip][vp_ip]["longitude"]= vp_lon

                # compute true distance between vp and target
                geoloc_results[target_ip][vp_ip]["calculated_d"] = distance(
                    vp_lat,
                    target_lat,
                    vp_lon,
                    target_lon
                )

                # compute measured distance based on rtt
                geoloc_results[target_ip][vp_ip]["measured_d"] = rtt_to_km(min_rtt)

                # debugging
                # print(
                #     f"target: {result['dst_addr']}: {anchors[target_ip]} \n \
                #     anchor: {vp_ip}: {anchors[vp_ip]} \n \
                #     mean rtt: {geoloc_results[target_ip][vp_ip]} \n \
                #     distance (calculated): {geoloc_results[target_ip][vp_ip]['calculated_d']} \n \
                #     distance (measured)  : {geoloc_results[target_ip][vp_ip]['measured_d']}"
                # )
            except TypeError:
                continue
        else:
            print(f"no results: {result}")



getting results for target ip: 207.226.224.236
getting results for target ip: 46.19.61.33
getting results for target ip: 196.6.220.42
getting results for target ip: 130.59.80.2
getting results for target ip: 77.88.216.1


In [2]:
# check
for target_ip in geoloc_results:
    print(f"number of responsive vp for {target_ip} : {len(geoloc_results[target_ip])}")

number of responsive vp for 207.226.224.236 : 15
number of responsive vp for 46.19.61.33 : 15
number of responsive vp for 196.6.220.42 : 15
number of responsive vp for 130.59.80.2 : 15
number of responsive vp for 77.88.216.1 : 15


In [3]:
from pathlib import Path
from copy import copy

from geoloc_imc_2023.helpers import polygon_centroid, haversine, circle_intersections
from geoloc_imc_2023.draw_results import draw_results

measurement_file = Path(".") / "../databases/test/10_probes_10_target.pickle"
probes_path = Path(".") / "../datasets/probe_atlas.pickle"

def select_best_guess_centroid(target_ip, measurement_results):
    """
    Find the best guess
    that is the location of the vantage point closest to the centroid.
    """
    probe_circles = {}
    for _, vp_ip in enumerate(measurement_results):
        if measurement_results[vp_ip]:
            lat = measurement_results[vp_ip]["latitude"]
            lon = measurement_results[vp_ip]["longitude"]
            rtt = measurement_results[vp_ip]["rtt"]

            # too inflated RTT means that measurement will not provide usefull info
            if isinstance(rtt, float):
                    probe_circles[vp_ip] = (
                        lat,
                        lon,
                        rtt,
                        None,
                        None,
                    )
                    print(f"target ip {target_ip}")
                    print(f"vp_anchor = {anchors[vp_ip]}")
                    print(f"probe_circle = {lat, lon,rtt}")
            else:
                # TODO: remove measurements were not RTT is available
                print(f"measurement rtt is not int : {rtt}")
    print(f"NUMBER OF MEASUREMENT RESULTS FOR TARGET IP: {target_ip} : {len(measurement_results)}")
    print()

    # draw circles
    if not probe_circles: return

    intersections = circle_intersections(probe_circles.values(), speed_threshold=4/9)
    print("intersecion:", intersections)
    target = (anchors[target_ip]["latitude"], anchors[target_ip]["longitude"])

    draw_results(copy(probe_circles), intersections, target)

    # for ip, circle in probe_circles.items():
    #     print(f"target ip {ip}: results : {circle}")

    if intersections:
        centroid = polygon_centroid(intersections)
        print("centroid:", centroid)
        target_description = anchors[target_ip]

        if centroid:
            for ip_src, circle in probe_circles.items():

                measured_d = haversine((circle[0], circle[1]), centroid)
                calculated_d = haversine((target_description["latitude"], target_description["longitude"]), centroid)

                print("ip_src:",ip_src,"dst (measured):", measured_d, "dst (calculated):", calculated_d)

for i, (target_ip, geoloc_result) in enumerate(geoloc_results.items()):
    select_best_guess_centroid(target_ip, geoloc_result)

target ip 207.226.224.236
vp_anchor = {'id': 6413, 'is_anchor': True, 'country_code': 'DE', 'latitude': 50.1385, 'longitude': 8.7375}
probe_circle = (50.1385, 8.7375, 0.922516)
target ip 207.226.224.236
vp_anchor = {'id': 6433, 'is_anchor': True, 'country_code': 'DE', 'latitude': 50.1385, 'longitude': 8.7385}
probe_circle = (50.1385, 8.7385, 1.026076)
target ip 207.226.224.236
vp_anchor = {'id': 6534, 'is_anchor': True, 'country_code': 'DE', 'latitude': 50.1095, 'longitude': 8.6785}
probe_circle = (50.1095, 8.6785, 1.116488)
target ip 207.226.224.236
vp_anchor = {'id': 6558, 'is_anchor': True, 'country_code': 'DE', 'latitude': 50.1115, 'longitude': 8.7395}
probe_circle = (50.1115, 8.7395, 1.086811)
target ip 207.226.224.236
vp_anchor = {'id': 6561, 'is_anchor': True, 'country_code': 'DE', 'latitude': 50.1175, 'longitude': 8.7405}
probe_circle = (50.1175, 8.7405, 0.59629)
target ip 207.226.224.236
vp_anchor = {'id': 6601, 'is_anchor': True, 'country_code': 'DE', 'latitude': 50.1375, 'lo

target ip 46.19.61.33
vp_anchor = {'id': 6413, 'is_anchor': True, 'country_code': 'DE', 'latitude': 50.1385, 'longitude': 8.7375}
probe_circle = (50.1385, 8.7375, 1.596014)
target ip 46.19.61.33
vp_anchor = {'id': 6433, 'is_anchor': True, 'country_code': 'DE', 'latitude': 50.1385, 'longitude': 8.7385}
probe_circle = (50.1385, 8.7385, 1.763706)
target ip 46.19.61.33
vp_anchor = {'id': 6534, 'is_anchor': True, 'country_code': 'DE', 'latitude': 50.1095, 'longitude': 8.6785}
probe_circle = (50.1095, 8.6785, 1.989033)
target ip 46.19.61.33
vp_anchor = {'id': 6558, 'is_anchor': True, 'country_code': 'DE', 'latitude': 50.1115, 'longitude': 8.7395}
probe_circle = (50.1115, 8.7395, 1.616209)
target ip 46.19.61.33
vp_anchor = {'id': 6561, 'is_anchor': True, 'country_code': 'DE', 'latitude': 50.1175, 'longitude': 8.7405}
probe_circle = (50.1175, 8.7405, 1.942369)
target ip 46.19.61.33
vp_anchor = {'id': 6601, 'is_anchor': True, 'country_code': 'DE', 'latitude': 50.1375, 'longitude': 8.7375}
probe

target ip 196.6.220.42
vp_anchor = {'id': 6053, 'is_anchor': True, 'country_code': 'ZA', 'latitude': -26.1415, 'longitude': 28.0095}
probe_circle = (-26.1415, 28.0095, 60.244426)
target ip 196.6.220.42
vp_anchor = {'id': 6090, 'is_anchor': True, 'country_code': 'QA', 'latitude': 25.2775, 'longitude': 51.5215}
probe_circle = (25.2775, 51.5215, 279.515445)
target ip 196.6.220.42
vp_anchor = {'id': 6224, 'is_anchor': True, 'country_code': 'TZ', 'latitude': -6.8105, 'longitude': 39.2885}
probe_circle = (-6.8105, 39.2885, 13.350908)
target ip 196.6.220.42
vp_anchor = {'id': 6358, 'is_anchor': True, 'country_code': 'UG', 'latitude': 0.3115, 'longitude': 32.5795}
probe_circle = (0.3115, 32.5795, 185.625027)
target ip 196.6.220.42
vp_anchor = {'id': 6431, 'is_anchor': True, 'country_code': 'SA', 'latitude': 22.3075, 'longitude': 39.1085}
probe_circle = (22.3075, 39.1085, 341.848858)
target ip 196.6.220.42
vp_anchor = {'id': 6569, 'is_anchor': True, 'country_code': 'ZA', 'latitude': -26.1385, '

target ip 130.59.80.2
vp_anchor = {'id': 6268, 'is_anchor': True, 'country_code': 'CH', 'latitude': 47.3915, 'longitude': 8.5205}
probe_circle = (47.3915, 8.5205, 1.265593)
target ip 130.59.80.2
vp_anchor = {'id': 6390, 'is_anchor': True, 'country_code': 'CH', 'latitude': 47.4275, 'longitude': 8.5615}
probe_circle = (47.4275, 8.5615, 0.713355)
target ip 130.59.80.2
vp_anchor = {'id': 6445, 'is_anchor': True, 'country_code': 'CH', 'latitude': 47.3995, 'longitude': 8.6195}
probe_circle = (47.3995, 8.6195, 1.083677)
target ip 130.59.80.2
vp_anchor = {'id': 6543, 'is_anchor': True, 'country_code': 'CH', 'latitude': 47.4485, 'longitude': 8.2095}
probe_circle = (47.4485, 8.2095, 1.302327)
target ip 130.59.80.2
vp_anchor = {'id': 6560, 'is_anchor': True, 'country_code': 'CH', 'latitude': 47.4275, 'longitude': 8.5605}
probe_circle = (47.4275, 8.5605, 0.873887)
target ip 130.59.80.2
vp_anchor = {'id': 6689, 'is_anchor': True, 'country_code': 'CH', 'latitude': 47.4115, 'longitude': 8.5415}
probe

measurement rtt is not int : *
target ip 77.88.216.1
vp_anchor = {'id': 6414, 'is_anchor': True, 'country_code': 'UA', 'latitude': 46.9795, 'longitude': 31.9875}
probe_circle = (46.9795, 31.9875, 13.9542)
target ip 77.88.216.1
vp_anchor = {'id': 6453, 'is_anchor': True, 'country_code': 'RO', 'latitude': 47.1585, 'longitude': 27.5895}
probe_circle = (47.1585, 27.5895, 51.458383)
measurement rtt is not int : *
target ip 77.88.216.1
vp_anchor = {'id': 6510, 'is_anchor': True, 'country_code': 'MD', 'latitude': 47.0175, 'longitude': 28.8375}
probe_circle = (47.0175, 28.8375, 47.373779)
measurement rtt is not int : *
target ip 77.88.216.1
vp_anchor = {'id': 6619, 'is_anchor': True, 'country_code': 'UA', 'latitude': 48.0015, 'longitude': 37.8375}
probe_circle = (48.0015, 37.8375, 98.249298)
target ip 77.88.216.1
vp_anchor = {'id': 6845, 'is_anchor': True, 'country_code': 'BG', 'latitude': 44.1195, 'longitude': 27.2595}
probe_circle = (44.1195, 27.2595, 51.457313)
measurement rtt is not int : 